In [ ]:
import sys
sys.path.append("..")
# make sure the requirements are installed if you are running this notebook locally
!pip install -r '../requirements.txt'

In [ ]:
# in case of crashes or import errors here, make sure your notebook is running a kernel with the correct python environment, and installed the requirements!
import roboflow
from datasetUtils import *
import numpy as np

In [ ]:
#use display function from utils to display random images from the dataset

#run as many times as you wish, selection is random among the dataset's 'valid' subset
display_test_image('../mergeDataset')
